In [18]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [19]:
# Add warehouse data
entry2_data = pd.read_csv("../entry_loc2.csv")
entry2_data = entry2_data.dropna()
entry2_data['HD entry'] = 1*(entry2_data['HD'] > 0)
entry2_data['LOW entry'] = 1*(entry2_data['LOW'] > 0)
entry2_data['log income'] = np.log(1 + entry2_data['income_per_capita'])
entry2_data['log population'] = np.log(1 + entry2_data['population'])
entry2_data['log hd warehouse'] = np.log(1 + entry2_data['hd warehouse distance'])
entry2_data['log low warehouse'] = np.log(1 + entry2_data['low warehouse distance'])
entry2_data = entry2_data[(entry2_data['population']>=10000) & (entry2_data['income_per_capita']>=15000)]
entry2_data = entry2_data[(entry2_data['lon'] <= -30) & (entry2_data['lat'] >= 25) ]
entry2_data = sm.add_constant(entry2_data)

In [31]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        p = np.zeros((2, 2, n))

        for a_j in [0, 1]:
            util1 = np.dot(self.exog[:, 0:k], params[0:k]) + params[-3]*a_j
            util2 = np.dot(self.exog[:, k:2*k],params[k:2*k]) + params[-2]*a_j
            p[0, a_j, :] = 1 / (1 + np.exp(util1))
            p[1, a_j, :] = 1 / (1 + np.exp(util2))
        
        #choose lambda
        lamb = .5
        delta_pos = 1*(params[-3] >= 0)*(params[-2] >= 0)
        delta_neg = 1*(params[-3] <= 0)*(params[-2] <= 0)

        if (delta_pos == 0) and (delta_neg == 0):
            params[-1] = 0
            
        #constrain delta2 to be pos...
        rho = np.clip(params[-1],-1,1)
        print(rho)
        
        # solve for probablity of nash
        prob01 = (p[0, 1, :])*(1 - p[1, 0, :]) 
        prob10 = (1 - p[0, 0, :])*(p[1, 1, :]) 
        prob00 = p[0, 0, :] * p[1, 0, :] 
        prob11 = (1 - p[0, 1, :])*(1 - p[1, 1, :])
        
        #introduce the copula
        prob01_rho = prob01*(1+rho*prob10)
        prob10_rho = prob10*(1+rho*prob01)
        prob00_rho = prob00*(1+rho*prob11)
        prob11_rho = prob11*(1+rho*prob00)
        
        #deal with multiple eq
        if delta_pos > 0 and delta_neg==0:
            print(delta_pos, params[-2],params[-3])
            pr_both = 1 - prob10_rho - prob01_rho
            pr_both_wrong = prob00_rho + prob11_rho
            prob00_rho = prob00_rho*pr_both/pr_both_wrong
            prob11_rho = prob11_rho*pr_both/pr_both_wrong
            
        if delta_neg > 0 and delta_pos==0:
            print(delta_pos, params[-2],params[-3])
            pr_both = 1 - prob11_rho - prob00_rho
            pr_both_wrong = prob10_rho + prob01_rho
            prob01_rho = prob01_rho*pr_both/pr_both_wrong
            prob10_rho = prob10_rho*pr_both/pr_both_wrong
            
        print(prob01_rho+prob10_rho+prob00_rho+prob11_rho)
        print('---')

        
        # compute empirical likelihood
        p00 = (1 - self.endog[:, 0])*(1 - self.endog[:, 1])
        p11 = self.endog[:, 0]*self.endog[:, 1]
        p10 = self.endog[:, 0] * (1 - self.endog[:, 1])
        p01 = (1 - self.endog[:, 0]) * self.endog[:, 1]
        


        ll = p00 * prob00_rho + p11 * prob11_rho + p01 * prob01_rho + p10 * prob10_rho
        
        return -np.log(np.maximum(ll, 1e-12))
        

    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        k = int(self.exog.shape[1]/2)
        x1 = np.concatenate( (self.exog[:, 0:k], self.endog[:,1].reshape(self.endog.shape[0],1) ) ,axis=1)
        x2 = np.concatenate( (self.exog[:, k:2*k], self.endog[:,0].reshape(self.endog.shape[0],1) ),axis=1)
        params1 =  sm.Logit(self.endog[:, 0], x1).fit().params
        params2 = sm.Logit(self.endog[:, 1], x2).fit().params
        start_params = np.concatenate((params1[0:-1],params2[0:-1],[-5,-5, 1] ))
        print(start_params)
        return super(NashLogit, self).fit(start_params=start_params, **kwds)

        
        
x = entry2_data[['const','log income','log population','log hd warehouse',
                 'const','log income','log population','log low warehouse']].copy()
y = entry2_data[['HD entry','LOW entry']]

NashLogit_model = NashLogit(y,x).fit()
NashLogit_model.summary()

Optimization terminated successfully.
         Current function value: 0.516223
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.521237
         Iterations 6
[-1.09364059e+01 -5.08298059e-03  9.45608270e-01  8.33899479e-03
  4.47877722e+00 -1.15220618e+00  5.79471773e-01  5.06419108e-03
 -5.00000000e+00 -5.00000000e+00  1.00000000e+00]
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.25
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.25 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.0 -5.0
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.045454545454545 -5.045454545454545
[1. 1. 1. ... 1. 1. 1.]
---
1.0
0 -5.05371

1 0.4041386357924328 0.7905686934251652
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.41599707512586437 1.9418877098253429
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.42043586240076913 0.5390908409152039
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.7315931080241045 0.46959417940836956
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.8611386750074677 0.5922395913444207
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.975046398706475 0.7143356969196341
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.3879160933830432 0.5936197504658673
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.9752569906477487 0.08649878355768625
[1. 1. 1. ... 1. 1. 1.]
---
0.0
[0.99045485 0.99011511 0.99176267 ... 0.96147073 0.95412293 0.9940138 ]
---
0.0
1 0.5641320520405464 0.9971217364989469
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.689727844028215 0.6014760686810139
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.067547826114306 0.7959633498892134
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.8057128580782061 0.6935703831078683
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 0.5734441812472437 0.8873922960250489
[1. 1. 1. ... 1

0.0
1 1.3701290838895424 0.006567136177837805
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.3333227915514447 0.006164204535714209
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.3768302899454632 0.0076589744609514175
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.3168387185069135 0.006569042392308563
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.3562882507772707 0.009232083206840394
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.356229327934149 0.011195706802445974
[1. 1. 1. ... 1. 1. 1.]
---
0.0
[0.99997319 0.99997227 0.99997726 ... 0.99990561 0.99990488 0.99999194]
---
0.0
[0.99989365 0.99988996 0.99990978 ... 0.99962256 0.999618   0.9999674 ]
---
0.0
1 1.352408074093049 0.01057555872454224
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.3538964696412759 0.014096476728738034
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.3983621891958185 0.007668872528132822
[1. 1. 1. ... 1. 1. 1.]
---
0.0
[0.99997869 0.99997794 0.99998193 ... 0.99992332 0.99992189 0.99999329]
---
0.0
1 1.3865759608314512 0.014392017634325373
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.35294691809524

1 1.2157432209809365 0.10013174269538311
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.245522875529261 0.09255401651832344
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.239972862943227 0.11232045355354556
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2322039640011089 0.06981003969558441
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2332946443499218 0.09499890185325881
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2285690527634774 0.08808261663393954
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2122636700148588 0.0852396405247125
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.1784687309418558 0.1092624633997005
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.1769470560335664 0.08798382252826947
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.125797849711109 0.0830343812568034
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2195658308445427 0.09247265758909828
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2156261407049032 0.09242820887708436
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.1524604234177542 0.11352328262071724
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2810696952380307 0.08790115221328382
[1. 1. 1. ... 1.

[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2075288048221577 0.09358895995449318
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2155605132494056 0.09309204293354889
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.202551396143279 0.09160508621646127
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2041351023068123 0.08939157428832079
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2047933756407938 0.09077646444405704
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2108552302593247 0.09206326409497786
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2052049695113896 0.08994183812313031
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2130273097351933 0.0881420655022293
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.211085123637309 0.08670476855586745
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.216743140273774 0.08202491205959717
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2168816676473897 0.08303077895735048
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2249066352225872 0.08847695529931462
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2157789617112198 0.08740264372680472
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2242705447954283 0.

1.220703125e-05
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
-1.220703125e-05
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
1.220703125e-05
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
-1.220703125e-05
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1. 1. 1.]
---
0.0
1 1.2636965306790833 0.014862207861623927
[1. 1. 1. ... 1.

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  NashLogit Results                                  
=====================================================================================
Dep. Variable:     ['HD entry', 'LOW entry']   Log-Likelihood:                -3181.7
Model:                             NashLogit   AIC:                             6373.
Method:                   Maximum Likelihood   BIC:                             6403.
Date:                       Thu, 01 Jul 2021                                         
Time:                               13:39:38                                         
No. Observations:                       2952                                         
Df Residuals:                           2947                                         
Df Model:                                  4                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -15.2170      1.623     -9.376      0.000     -18.398     -12.036
log income           -0.0053      0.143     -0.037      0.970      -0.286       0.276
log population        1.4007      0.086     16.349      0.000       1.233       1.569
log hd warehouse      0.0129      0.041      0.320      0.749      -0.066       0.092
const                 1.2652      1.539      0.822      0.411      -1.751       4.282
log income           -0.9400      0.143     -6.559      0.000      -1.221      -0.659
log population        0.6839      0.086      7.974      0.000       0.516       0.852
log low warehouse     0.0063      0.045      0.140      0.888      -0.082       0.095
par0                  0.0149      0.321      0.046      0.963      -0.614       0.644
par1                  1.2637      0.268      4.716      0.000       0.739       1.789
par2                       0      0.177          0      1.000      -0.348       0.348
=====================================================================================
"""